In [3]:
import sqlite3
import re
import calendar # for isleap
import datetime
import pandas as pd

In [41]:
class GncDate(object):
    class GMT1(datetime.tzinfo):
        def utcoffset(self, dt):
            return datetime.timedelta(hours=1) + self.dst(dt)
        
        def dst(self, dt):
            # DST starts last Sunday in March
            d = datetime.datetime(dt.year, 4, 1)   # ends last Sunday in October
            self.dston = d - datetime.timedelta(days=d.weekday() + 1)
            d = datetime.datetime(dt.year, 11, 1)
            self.dstoff = d - datetime.timedelta(days=d.weekday() + 1)
            if self.dston <=  dt.replace(tzinfo=None) < self.dstoff:
                return datetime.timedelta(hours=1)
            else:
                return datetime.timedelta(0)
            
        def tzname(self,dt):
            return "GMT +1"

    def __init__(self, isodate):
        self.date = None

        try:
            self.date = datetime.datetime.strptime(isodate, "%Y-%m-%d")
        except ValueError:
            try:
                self.date = datetime.datetime.strptime(isodate, "%y-%m-%d")
            except ValueError:
                try:
                    self.date = datetime.datetime.strptime(isodate, "%y-%m-%d%z")
                except ValueError:
                    self.date = datetime.datetime.strptime(isodate, "%Y-%m-%d%z")
    
        if self.date.tzinfo == None:
            gmt1 = self.GMT1()
            self.date = self.date.replace(tzinfo=gmt1)
    
    def to_gnc(self):
        d = self.date
        d = d - d.utcoffset()
        ret = "{:04}{:02}{:02}{:02}{:02}{:02}".format(d.year, d.month, d.day, d.hour, d.minute, d.second)
        return ret
    
    def from_gnc(gncdate_utc):
        if len(gncdate_utc) != 14:
            raise ValueError("Invalid gnucash-date: length != 14")
        date_utc = datetime.datetime.strptime(gncdate_utc, "%Y%m%d%H%M%S")
        time = date_utc.time()
        if time.second != 0:
            raise ValueError("Invalid gnucash-date: seconds != 00")
        if time < datetime.time(12,0,0):
            # we're in a negative timezone UTC-1to-12
            utc_offset = datetime.timedelta(hours=time.hour, minutes=time.minute)
            tzstr = "-{:02}{:02}".format(time.hour, time.minute)
            date_tz = date_utc - utc_offset
        else:
            # we're in a positive timezone
            utc_offset = datetime.timedelta(hours=23) - datetime.timedelta(hours=time.hour, minutes=time.minute) + datetime.timedelta(hours=1)
            tzstr = "+{:02}{:02}".format(utc_offset.seconds // 3600, (utc_offset.seconds % 3600)//60)
            date_tz = date_utc + utc_offset
        return GncDate("%s-%s-%s%s" % (date_tz.year, date_tz.month, date_tz.day, tzstr))

In [42]:
from unittest import *

class TestGncDate(TestCase):
    def test_GncDateFromIsoString_noTz(self):
        tests = [{'teststr': "2017-03-28", 'y':2017, 'm':3,  'd':28},
                 {'teststr': "2015-02-01", 'y':2015, 'm':2,  'd':1},
                 {'teststr': "2016-02-29", 'y':2016, 'm':2,  'd':29}, # leap
                 {'teststr': "1999-01-15", 'y':1999, 'm':1,  'd':15},
                 {'teststr': "2010-11-13", 'y':2010, 'm':11, 'd':13},
                 {'teststr': "2100-12-31", 'y':2100, 'm':12, 'd':31},
                 {'teststr': "15-12-28",   'y':2015, 'm':12, 'd':28},
                 {'teststr': "99-02-12",   'y':1999, 'm':2,  'd':12},
                 {'teststr': "01-04-10",   'y':2001, 'm':4,  'd':10},
                 {'teststr': "14-03-30",   'y':2014, 'm':3,  'd':30},
                 {'teststr': "00-06-01",   'y':2000, 'm':6,  'd':1}]
        
        for t in tests:
            with self.subTest(ts = t['teststr']):
                d = GncDate(t['teststr'])
                self.assertIsInstance(d.date, datetime.datetime)
                self.assertEqual(d.date.year, t['y'])
                self.assertEqual(d.date.month, t['m'])
                self.assertEqual(d.date.day, t['d'])
                self.assertIsNotNone(d.date.tzinfo)
                self.assertEqual(d.date.tzinfo.tzname(d), "GMT +1")

        invalid_input = ["03-28-2017",
                         "13-2017-01"
                         "2017-03-32",
                         "2017-13-01",
                         "13-01-2017",
                         "2017-03-32",
                         "2017.01.03",
                         "17.01.03",
                         "17-00-28",
                         "17-13-28",
                         "17-03-32",
                         "17-02-29"] # 2017 is non-leap
        
        for inv in invalid_input:
            with self.subTest(inv = inv):
                with self.assertRaises(ValueError):
                    d = GncDate(inv)

        with self.assertRaises(TypeError):
            d = GncDate(2017)
            

    def test_GncDateFromIsoString_withTz(self):
        tests = [{'teststr': "2017-03-28+0100", 'y':2017, 'm':3,  'd':28, 'utc_h':1,   'utc_m':0},
                 {'teststr': "2015-02-01-2315", 'y':2015, 'm':2,  'd':1,  'utc_h':-23, 'utc_m':-15},
                 {'teststr': "2012-02-29+2315", 'y':2012, 'm':2,  'd':29, 'utc_h':23,  'utc_m':15}, # leap year
                 {'teststr': "1999-01-15-1200", 'y':1999, 'm':1,  'd':15, 'utc_h':-12, 'utc_m':0},
                 {'teststr': "2010-11-13+0000", 'y':2010, 'm':11, 'd':13, 'utc_h':0,   'utc_m':0},
                 {'teststr': "2100-12-31-0000", 'y':2100, 'm':12, 'd':31, 'utc_h':0,   'utc_m':0},
                 {'teststr': "15-12-28+0100",   'y':2015, 'm':12, 'd':28, 'utc_h':1,   'utc_m':0},
                 {'teststr': "99-02-12-0335",   'y':1999, 'm':2,  'd':12, 'utc_h':-3,  'utc_m':-35},
                 {'teststr': "01-04-10+1200",   'y':2001, 'm':4,  'd':10, 'utc_h':12,  'utc_m':0},
                 {'teststr': "14-03-30+0001",   'y':2014, 'm':3,  'd':30, 'utc_h':0,   'utc_m':1},
                 {'teststr': "00-06-01-0100",   'y':2000, 'm':6,  'd':1,  'utc_h':-1,  'utc_m':0}]
        
        for t in tests:
            with self.subTest(ts = t['teststr']):
                d = GncDate(t['teststr'])
                self.assertIsInstance(d.date, datetime.datetime)
                self.assertEqual(d.date.year, t['y'])
                self.assertEqual(d.date.month, t['m'])
                self.assertEqual(d.date.day, t['d'])
                self.assertIsNotNone(d.date.tzinfo)
                self.assertEqual(d.date.tzinfo.utcoffset(d.date), 
                                 datetime.timedelta(hours=t['utc_h'], minutes=t['utc_m']))
                
        invalid_input = ["03-28-2017+0100",
                         "2017-03-32+0100",
                         "2017-02-29+0100", # 2017 is non-leap
                         "2017-03-32+0100",
                         "2017-03-28Z0100",
                         "2017-03-28z0100",
                         "2017-03-28Z+100",
                         "2017-03-28+01:00",
                         "2017-03-28+01:00",
                         "2017-03-28+01",
                         "17-03-28Z+0100",
                         "17-13-28+0100",
                         "17-03-32+0100"]
        
        for inv in invalid_input:
            with self.subTest(inv = inv):
                with self.assertRaises(ValueError):
                    d = GncDate(inv)
    
    def test_GncDate_toGnc(self):
        tests = [{'teststr': "2017-03-28+0100", 'gnc': "20170327230000"},
                 {'teststr': "2015-02-01-2315", 'gnc': "20150201231500"},
                 {'teststr': "2012-02-29+2315", 'gnc': "20120228004500"}, # leap year
                 {'teststr': "1999-01-15-1200", 'gnc': "19990115120000"},
                 {'teststr': "2010-11-13+0000", 'gnc': "20101113000000"},
                 {'teststr': "2100-12-31-0000", 'gnc': "21001231000000"},
                 {'teststr': "15-12-28+0100",   'gnc': "20151227230000"},
                 {'teststr': "99-02-12-0335",   'gnc': "19990212033500"},
                 {'teststr': "01-04-10+1200",   'gnc': "20010409120000"},
                 {'teststr': "14-03-30+0001",   'gnc': "20140329235900"},
                 {'teststr': "00-01-01+0100",   'gnc': "19991231230000"},
                 {'teststr': "16-03-01+0200",   'gnc': "20160229220000"},
                 {'teststr': "16-03-01",        'gnc': "20160229230000"},  # GMT+1 / DST off
                 {'teststr': "16-05-01",        'gnc': "20160430220000"},] # GMT+1 / DST on
        
        for t in tests:
            with self.subTest(ts = t['teststr']):
                d = GncDate(t['teststr'])
                self.assertEqual(d.to_gnc(), t['gnc'])
                
    def test_GncDate_fromGnc(self):
        tests = [{'date': "2017-03-28+0100", 'gnc': "20170327230000"},
                 {'date': "2015-02-01-2315", 'gnc': "20150201231500"},
                 {'date': "2012-02-29+2315", 'gnc': "20120228004500"}, # leap year
                 {'date': "1999-01-15-1200", 'gnc': "19990115120000"},
                 {'date': "2010-11-13+0000", 'gnc': "20101113000000"},
                 {'date': "2100-12-31-0000", 'gnc': "21001231000000"},
                 {'date': "15-12-28+0100",   'gnc': "20151227230000"},
                 {'date': "99-02-12-0335",   'gnc': "19990212033500"},
                 {'date': "01-04-10+1200",   'gnc': "20010409120000"},
                 {'date': "14-03-30+0001",   'gnc': "20140329235900"},
                 {'date': "00-01-01+0100",   'gnc': "19991231230000"},
                 {'date': "16-03-01+0200",   'gnc': "20160229220000"},
                 {'date': "16-03-01",        'gnc': "20160229230000"},  # GMT+1 / DST off
                 {'date': "16-05-01",        'gnc': "20160430220000"},] # GMT+1 / DST on
        
        for t in tests:
            with self.subTest(gnc = t['gnc']):
                d = GncDate.from_gnc(t['gnc'])
                self.assertEqual(d.date, GncDate(t['date']).date)
                self.assertEqual(d.to_gnc(), t['gnc'])
                
        invalid_input = ["2016043022000",
                         "201604302200000",
                         "20160430220010",
                         "20160432220000",
                         "20160430250000",
                         "20160430226000",
                         "2016043022000a",
                         "20161430220000",
                         "2016043022a000",
                         "20160400220000",
                         "20160030000000",
                         "00000430220000",
                         "20160430260000"]
        
        for inv in invalid_input:
            with self.subTest(inv = inv):
                with self.assertRaises(ValueError):
                    d = GncDate.from_gnc(inv)
                             
a = TestGncDate()
suite = TestLoader().loadTestsFromModule(a)
TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.017s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [472]:
from functools import wraps
import inspect

def get_class_that_defined_method(meth):
    if inspect.ismethod(meth):
        for cls in inspect.getmro(meth.__self__.__class__):
            if cls.__dict__.get(meth.__name__) is meth:
                return cls
        meth = meth.__func__ # fallback to __qualname__ parsing
    if inspect.isfunction(meth):
        cls = getattr(inspect.getmodule(meth),
                      meth.__qualname__.split('.<locals>', 1)[0].rsplit('.', 1)[0])
        if isinstance(cls, type):
            return cls
        
def unimplemented(func):
    @wraps(func)
    def tmp(*args, **kwargs):
        #func_name = sys._getframe().f_code.co_name
        #func_name = inspect.currentframe().f_code.co_name
        #func_name = inspect.stack()[0][0].f_code.co_name
        #func_name = inspect.stack()[0][3]
        caller = inspect.stack()[1][3]
        cls = get_class_that_defined_method(func)
        print("Warning: Unimplemented function %s() called by %s:%s()" % (func.__name__, cls, caller))
        return func(*args, **kwargs)
    return tmp

class Account(object):
    ROOT = 1
    ASSET = 2
    CASH = 3
    BANK = 4
    LIABILITY = 5
    CREDIT = 6
    TRADING = 7
    
    def __init__(self, book, accDF, parent):
        acc = accDF.iloc[0]
        self.book = book
        self.guid = acc.guid
        self.name = acc['name']
        self.type = self._type(acc.account_type)
        self.parent_guid = acc.parent_guid
        self.parent = parent
        self.description = acc.description
        self.commodity = self.book.getCommodityByGuid(acc.commodity_guid)
        self.hidden = acc.hidden
        self.placeholder = (acc.placeholder == 1)
        self.children = [] # to be completed via addChild
        self.hier_name = self._calcHierName()
        if parent is not None:
            parent.addChild(self)
        
    def _type(self, accType):
        if accType == 'ROOT': return self.ROOT
        if accType == 'ASSET': return self.ASSET
        if accType == 'CASH': return self.ROOT
        if accType == 'BANK': return self.BANK
        if accType == 'LIABILITY': return self.LIABILITY
        if accType == 'CREDIT': return self.CREDIT
        if accType == 'TRADING': return self.TRADING
        
    def _calcHierName(self):
        name = ""
        cur = self
        name += cur.name
        while cur.parent:
            if cur.parent.parent == None:
                # Parent is root -> so exit
                break
            cur = cur.parent
            name = cur.name + ":" + name
        return name
        
    def addChild(self, child):
        if not isinstance(child, Account):
            ValueError("Non Account-type passed for child")
        self.children.append(child)
        
class Commodity(object):
    CURRENCY = 1
    def __init__(self, book, commDF):
        comm = commDF.iloc[0]
        self.book = book
        self.guid = comm.guid
        self.name = comm.mnemonic
        self.fullname = comm.fullname
        self.type = self._type(comm.namespace)
        self.iscurrency = comm.namespace == "CURRENCY"
        
    def _type(self, commType):
        if commType == "CURRENCY": return self.CURRENCY
    
    
class Book(object):
    def __init__(self, gnc_db):
        self.db = sqlite3.connect(gnc_db)
        self._getCommodities()
        self._createCommodityObjects()
        self._getBookRootAccount()
        self._getBookAccounts()
        self._createAccountObjects()
        self._getCommodityPrices()
        self._getTransactions()
        self._getSplits()
    
    def _getCommodities(self):
        self.commDF = pd.read_sql_query("SELECT * FROM commodities;", self.db)
        
    def _createCommodityObjects(self):
        self.commodities = []
        for i in range(self.commDF.shape[0]):
            comm = self.commDF.iloc[[i]]
            c = Commodity(self, comm)
            self.commodities.append(c)
            
    def _getCommodityPrices(self):
        self.pricesDF = pd.read_sql_query("SELECT * FROM prices;", self.db)
                
    def _getBookRootAccount(self):
        """Obtain the root account guid of the gnucash book.
        
        Assumptions:
        - There is only one root account in the book
        """
        r = self.db.execute("SELECT root_account_guid FROM books;").fetchone()
        if r is None:
            raise ValueError("No valid root_account found in the gnucash book")
        else:
            self.rootAccountGuid = r[0]

    def _getBookAccounts(self):
        self.accDF = pd.read_sql_query("SELECT * FROM accounts;", self.db)
        
    def _createAccountObjects(self):
        self.root = Account(self, self.accDF.loc[self.accDF.guid == self.rootAccountGuid], parent=None)
        self.accounts = [self.root]
        for child, parentGuid in self._iterAccountTree(self.root.guid):
            acc = Account(self, child, self.getAccountByGuid(parentGuid))
            self.accounts.append(acc)

    def _iterAccountTree(self, parentGuid):
        children = self._findChildrenOf(parentGuid)
        for i in range(children.shape[0]):
            child = children.iloc[[i]]
            yield (child, parentGuid)
            yield from self._iterAccountTree(child.iloc[0].guid)
        
    def _findChildrenOf(self, parentGuid):
        ch = self.accDF.loc[self.accDF.parent_guid == parentGuid]
        return ch
    
    def _getTransactions(self):
        self.txDF = pd.read_sql_query("SELECT * FROM transactions;", self.db)
    
    def _getSplits(self):
        self.splitsDF = pd.read_sql_query("SELECT * FROM splits;", self.db)
    
    def getAccountByGuid(self, accGuid):
        for a in self.accounts:
            if a.guid == accGuid:
                return a
        return None
    
    def getAccountByName(self, accName, strict=True):
        for a in self.accounts:
            if a.name == accName:
                return a
        return None
    
    def getCommodityByGuid(self, commGuid):
        for c in self.commodities:
            if c.guid == commGuid:
                return c
        return None
    
    def getCommodityByName(self, commName):
        for c in self.commodities:
            if c.name == commName:
                return c
        return None
    
    def getCommodityPrice(self, comm, curr, refDate, useInverseRate=False):
        if not self._isGuid(comm):
            comm = self.getCommodityByName(comm).guid
        if not self._isGuid(curr):
            curr = self.getCommodityByName(curr).guid
        
        # Find and sort prices of the required commodity
        # Search for records where commodity is both in base and quote position
        prices = self.pricesDF.query("(commodity_guid == '%s' and currency_guid == '%s')" % 
                                     (comm, curr))
        if useInverseRate:
            prices = prices.append(self.pricesDF.query("(commodity_guid == '%s' and currency_guid == '%s')" % 
                                                    (curr, comm)))
        prices = prices.sort_values("date", axis=0)
        
        # Find closest by date price entry
        pred_price_df = prices.query("date <= '%s'" % (refDate.to_gnc()))
        succ_price_df = prices.query("date > '%s'" % (refDate.to_gnc()))
        best_price_df = None
        if pred_price_df.empty:
            best_price_df = succ_price_df.iloc[0]
        if succ_price_df.empty:
            best_price_df = pred_price_df.iloc[-1]
        if best_price_df is None:
            pred_date = datetime.datetime.strptime(pred_price_df.iloc[-1].date, "%Y%m%d%H%M%S")
            succ_date = datetime.datetime.strptime(succ_price_df.iloc[0].date, "%Y%m%d%H%M%S")
            ref_date = datetime.datetime.strptime(refDate.to_gnc(), "%Y%m%d%H%M%S")

            if (ref_date - pred_date) > (succ_date - ref_date):
                best_price_df = succ_price_df.iloc[0]
            else:
                best_price_df = pred_price_df.iloc[-1]

        if best_price_df.commodity_guid == comm:
            # Commodity is in base position
            return (best_price_df.value_num, best_price_df.value_denom)
        else:
            # Commodity is in quote position
            return (best_price_df.value_denom, best_price_df.value_num)
            
    def _getTxByQuery(self, queryString):
        res = self.txDF.query(queryString)
        return res
    
    def _getSplitsByTx(self, txGuid):
        res = self.splitsDF[self.splitsDF.tx_guid == txGuid]
        return res
    
    def _isGuid(self, guid):
        if not isinstance(guid, str):
            return False
        if not re.match(r"[0-9a-fA-F]{32}", guid):
            return False
        return True
    
    def _printAccounts(self, parent, level=0):
        print(level*" " + parent.name)
        for c in parent.children:
            self._printAccounts(c, level+3)
            
    def reportCf(self, accounts, per_beg, per_end, currency="CHF"):
        """Get total cashflow for a set of accounts.

        Algorithm:
            tx_of_period = get_tx_for_period(fiscal_period)
            splits = []
            for tx in tx_period:
                # retrieve all splits of the given transaction
                tx_splits = get_tx_splits(tx)

                # delete all splits pertaining to the accounts in questions
                tx_splits = del_splits_for_acc(accounts) 

                if no splits have been deleted:
                    continue with the next tx        

                if tx_splits is not empty:
                    if the given transaction is not in the desired currency:
                        get the closest prior exchange rate for the tx currency
                        convert all tx values to the desired currency
                    splits.append(tx_splits)
        """
        if isinstance(per_beg, str):
            per_beg = GncDate(per_beg)
        if per_end == "" or per_end is None:
            per_end = "2100-01-01"
        if isinstance(per_end, str):
            per_end = GncDate(per_end)
        if per_end.date < per_beg.date:
            raise ValueError("Period start is > period end")
            
        # Convert the account names to guid's
        accGuids = list(map(lambda x: self.getAccountByName(x).guid, accounts))
        
        # Get the guids of all trading accounts
        tradingGuids = list(map(lambda x: x.guid, filter(lambda x: x.type == Account.TRADING, book.accounts)))
        
        # Get report's currency guid
        currGuid = self.getCommodityByName(currency).guid
        
        # Get all tx in the given period
        tx_for_period = self._getTxByQuery('post_date >= "%s" and post_date <= "%s"' % 
                                           (per_beg.to_gnc(), per_end.to_gnc()))
        if tx_for_period.empty:
            return None
        
        # Empty dataframe that will accumulate the final account-trimmed splits
        splits = pd.DataFrame()
        
        # Dictionary for storing exchange rates
        fx_rates = {}
        fx_rates['base'] = currency
    
        #print(tx_for_period)
        for _, tx in tx_for_period.iterrows():
            #print(tx)
            tx_splits = self._getSplitsByTx(tx.guid)
        
            # Delete splits belonging to the accounts in question
            nb_tx_splits = tx_splits.size;
            tx_splits = tx_splits[~tx_splits.account_guid.isin(accGuids)]
            
            # If no splits have been deleted, the tx is to be dumped
            # as it does not concern the required accounts
            if tx_splits.size == nb_tx_splits:
                continue

            # Delete all splits belonging to TRADING accounts
            tx_splits = tx_splits[~tx_splits.account_guid.isin(tradingGuids)]

            if tx_splits.empty:
                continue
            
            # Apply exchange rate if needed
            if tx.currency_guid != currGuid:
                tx_curr = self.getCommodityByGuid(tx.currency_guid)
                if tx_curr.name in fx_rates:
                    fx_num, fx_denom = fx_rates[tx_curr.name]
                else:
                    fx_num, fx_denom = self.getCommodityPrice(tx.currency_guid, 
                                                              currGuid, 
                                                              per_end)
                    fx_rates[tx_curr.name] = (fx_num, fx_denom)
                tx_splits.value_num *= fx_num
                tx_splits.value_denom *= fx_denom
                tx.currency_guid = currGuid

            splits = splits.append(tx_splits)
            
        splits['value'] = splits['value_num'] / splits['value_denom']
        splits['quantity'] = splits['quantity_num'] / splits['quantity_denom']
        splits['account'] = splits.apply(lambda x: self.getAccountByGuid(x.account_guid).hier_name, axis=1)
        
        cleaned_splits = splits[['account', 'value', 'quantity', 'account_guid', 'tx_guid']]
        cf_sources = cleaned_splits.query("value < 0").copy()
        cf_sinks = cleaned_splits.query("value > 0").copy()
        cf_sources[['value', 'quantity']] *= -1
        
        res = {}
        res['gnc_period_begin'] = per_beg.to_gnc()
        res['gnc_period_end'] = per_end.to_gnc()
        res['raw_tx'] = tx_for_period
        res['raw_splits'] = splits
        res['fx_rates'] = fx_rates
        res['cf_sources'] = cf_sources
        res['sources_saldo'] = res['cf_sources'].groupby(by="account").sum().sort_index()
        res['cf_sinks'] = cf_sinks
        res['sinks_saldo'] = res['cf_sinks'].groupby(by="account").sum().sort_index()
        res['cf_total_inflow'] = cf_sources.value.sum()
        res['cf_total_outflow'] = cf_sinks.value.sum()
        res['period_saldo'] = res['cf_total_inflow'] - res['cf_total_outflow']
    
        return res


In [462]:
book = Book("golden-gnc.sqlite3")

In [470]:
from unittest import *

class TestGnucashBook(TestCase):
    def setUp(self):
        self.book = Book("golden-gnc.sqlite3")
        
    def test_getCommodityPrice(self):
        tests = [{'comm':"BGN", 'curr':"CHF", 'date':"2017-01-28", 'inv':True, 'fx':(10000, 17051)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2017-02-28", 'inv':True, 'fx':(10000, 17049)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2017-03-28", 'inv':True, 'fx':(10000, 17057)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-31", 'inv':True, 'fx':(546700000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-25", 'inv':True, 'fx':(547600000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-24", 'inv':True, 'fx':(547600000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-23", 'inv':True, 'fx':(547600000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2019-12-23", 'inv':True, 'fx':(553000000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2000-12-23", 'inv':True, 'fx':(5, 8)},
                 
                 {'comm':"BGN", 'curr':"CHF", 'date':"2017-01-28", 'inv':False, 'fx':(546700000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2017-02-28", 'inv':False, 'fx':(546700000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2017-03-28", 'inv':False, 'fx':(554500000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-31", 'inv':False, 'fx':(546700000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-25", 'inv':False, 'fx':(547600000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-24", 'inv':False, 'fx':(547600000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2016-12-23", 'inv':False, 'fx':(547600000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2019-12-23", 'inv':False, 'fx':(553000000, 1000000000)},
                 {'comm':"BGN", 'curr':"CHF", 'date':"2000-12-23", 'inv':False, 'fx':(7500, 12149)}]
        
        for t in tests:
            with self.subTest(d = t['date'], inv = t['inv']):
                fx = self.book.getCommodityPrice(t['comm'], t['curr'], GncDate(t['date']), t['inv'])
                self.assertEqual(t['fx'], fx)
    
suite = TestLoader().loadTestsFromTestCase(TestGnucashBook)
TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.522s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [473]:
from unittest import *

class IntegrTestBook(TestCase):
    def setUp(self):
        self.book = Book("golden-gnc.sqlite3")
        
    def test_CashFlowReport_Assets_Feb2017(self):
        accs_cf = ['Private', 'Private-Bills', "Epay Microaccount", "SGS credit account", 'Cash CHF']
        res = self.book.reportCf(accs_cf, GncDate("2017-01-28"), GncDate("2017-02-27"))
        self.assertEqual(res['gnc_period_begin'], '20170127230000')
        self.assertEqual(res['gnc_period_end'],   '20170226230000')
        self.assertEqual(res['cf_total_inflow'], 8397.70)
        self.assertEqual(res['cf_total_outflow'], 8537.406796999998)
        self.assertEqual(res['period_saldo'], -139.7067969999971)
        self.assertEqual(res['fx_rates'], {'BGN': (546700000, 1000000000), 'EUR': (10801, 10000), 'base': 'CHF'})
        self.assertEqual(res['sinks_saldo'].value.tolist(), [54.45, 138.45, 39.171054999999996, 43.380645, 505.9, 13.0, 94.4, 60.0, 405.0, 5.94, 70.3, 5.0, 20.0, 25.0, 172.8, 68.1, 647.66, 610.0, 897.6500000000001, 152.8, 84.2, 60.0, 98.75, 1750.0, 60.49203, 34.0, 98.45, 57.2, 57.75, 215.0, 42.3, 40.0, 1700.0, 210.263067])
        self.assertEqual(res['sinks_saldo'].quantity.tolist(), [50.0, 138.45, 71.65, 79.35, 505.9, 13.0, 94.4, 60.0, 405.0, 5.94, 70.3, 5.0, 20.0, 25.0, 172.8, 68.1, 647.66, 610.0, 897.6500000000001, 152.8, 84.2, 60.0, 98.75, 1750.0, 60.49, 34.0, 98.45, 57.2, 57.75, 215.0, 42.3, 40.0, 1700.0, 194.67])
        self.assertEqual(res['sinks_saldo'].index.tolist(), ['Assets:Current Assets:Cash EUR', 'Expenses:Alcohol', 'Expenses:BG:Bills BG', 'Expenses:BG:Tax BG', 'Expenses:Baby:Babysitting', 'Expenses:Baby:Clothes', 'Expenses:Baby:Toys', 'Expenses:Car:Gas', 'Expenses:Car:State Fees', 'Expenses:Clothes & Shoes', 'Expenses:Cosmetics', 'Expenses:Decoration', 'Expenses:Equipment', 'Expenses:Fun:Games', 'Expenses:Fun:Go out', 'Expenses:Gifts:Gifts family', 'Expenses:Groceries', 'Expenses:Horse', 'Expenses:Insurance:Health Insurance', 'Expenses:Luxury', 'Expenses:Medical', 'Expenses:Memberships', 'Expenses:Pub', 'Expenses:Rent', 'Expenses:Services', 'Expenses:State', 'Expenses:Supplies', 'Expenses:Transport:Public Transportation', 'Expenses:Utilities:Cable', 'Expenses:Utilities:Electricity', 'Expenses:Utilities:Garbage', 'Expenses:Utilities:Phone', 'Liabilities:Credit Card CFF Visa', 'Liabilities:Loan car'])
        self.assertEqual(res['sources_saldo'].value.tolist(), [221.1, 8176.6])
        self.assertEqual(res['sources_saldo'].quantity.tolist(), [221.1, 8176.6])
        self.assertEqual(res['sources_saldo'].index.tolist(), ['Expenses:Medical', 'Income:Salary'])
        
    def test_CashFlowReport_Assets_Mar2017(self):
        accs_cf = ['Private', 'Private-Bills', "Epay Microaccount", "SGS credit account", 'Cash CHF']
        res = self.book.reportCf(accs_cf, GncDate("2017-02-28"), GncDate("2017-03-27"))
        self.assertEqual(res['gnc_period_begin'], '20170227230000')
        self.assertEqual(res['gnc_period_end'],   '20170326220000')
        self.assertEqual(res['cf_total_inflow'], 8371.300000000001)
        self.assertEqual(res['cf_total_outflow'], 8676.613406999997)
        self.assertEqual(res['period_saldo'], -305.313406999996)
        self.assertEqual(res['fx_rates'], {'BGN': (554500000, 1000000000), 'EUR': (10821, 10000), 'base': 'CHF'})
        self.assertEqual(res['sinks_saldo'].value.tolist(), [85.65000000000002, 49.66102, 37.94998, 505.9, 9.0, 32.9, 126.05, 21.0, 63.0, 14.5, 40.0, 18.95, 26.25, 56.699999999999996, 78.0, 660.5999999999999, 500.0, 901.85, 441.65, 114.39999999999999, 1750.0, 128.6, 73.95, 1730.0, 39.05, 112.75, 57.6, 40.0, 750.0, 210.652407])
        self.assertEqual(res['sinks_saldo'].quantity.tolist(), [85.65000000000002, 89.56, 68.44, 505.9, 9.0, 32.9, 126.05, 21.0, 63.0, 14.5, 40.0, 18.95, 26.25, 56.699999999999996, 78.0, 660.5999999999999, 500.0, 901.85, 441.65, 114.39999999999999, 1750.0, 128.6, 73.95, 1730.0, 39.05, 112.75, 57.6, 40.0, 750.0, 194.67])
        self.assertEqual(res['sinks_saldo'].index.tolist(), ['Expenses:Alcohol', 'Expenses:BG:Bills BG', 'Expenses:BG:Tax BG', 'Expenses:Baby:Babysitting', 'Expenses:Baby:Clothes', 'Expenses:Baby:Cosmetics', 'Expenses:Baby:Medical', 'Expenses:Baby:Toys', 'Expenses:Car:Gas', 'Expenses:Car:Parking', 'Expenses:Car:State Fees', 'Expenses:Cosmetics', 'Expenses:Decoration', 'Expenses:Equipment', 'Expenses:Fun:Go out', 'Expenses:Groceries', 'Expenses:Horse', 'Expenses:Insurance:Health Insurance', 'Expenses:Medical', 'Expenses:Pub', 'Expenses:Rent', 'Expenses:Services', 'Expenses:State', 'Expenses:State:Taxes', 'Expenses:Supplies', 'Expenses:Utilities:Billag', 'Expenses:Utilities:Cable', 'Expenses:Utilities:Phone', 'Liabilities:Credit Card CFF Visa', 'Liabilities:Loan car'])
        self.assertEqual(res['sources_saldo'].value.tolist(), [113.45, 8257.85])
        self.assertEqual(res['sources_saldo'].quantity.tolist(), [113.45, 8257.85])
        self.assertEqual(res['sources_saldo'].index.tolist(), ['Expenses:Medical', 'Income:Salary'])
        
    def test_CashFlowReport_ExpensesShoes(self):
        accs_cf = ['Clothes & Shoes']
        res = self.book.reportCf(accs_cf, GncDate("2016-09-28"), GncDate("2017-04-27"))
        self.assertEqual(res['gnc_period_begin'], '20160927220000')
        self.assertEqual(res['gnc_period_end'],   '20170426220000')
        self.assertEqual(res['cf_total_inflow'], 1786.9379999999999)
        self.assertEqual(res['cf_total_outflow'], 395.4600000000001)
        self.assertEqual(res['period_saldo'], 1391.4779999999998)
        self.assertEqual(res['fx_rates'], {'base': 'CHF', 'EUR': (108660000, 100000000)})
        self.assertEqual(res['sinks_saldo'].value.tolist(), [98.9, 9.9, 88.4, 39.8, 22.2, 2.7, 109.86000000000001, 23.7])
        self.assertEqual(res['sinks_saldo'].quantity.tolist(), [98.9, 9.9, 88.4, 39.8, 22.2, 2.7, 109.86000000000001, 23.7])
        self.assertEqual(res['sinks_saldo'].index.tolist(), ['Assets:Current Assets:PostFinance:Private-Bills', 'Expenses:Alcohol', 'Expenses:Baby:Clothes', 'Expenses:Baby:Toys', 'Expenses:Cosmetics', 'Expenses:Equipment', 'Expenses:Groceries', 'Expenses:Hobbies, Sport, Tourism'])
        self.assertEqual(res['sources_saldo'].value.tolist(), [32.598, 368.2, 301.95, 1084.19])
        self.assertEqual(res['sources_saldo'].quantity.tolist(),  [30.0, 368.2, 301.95, 1084.19])
        self.assertEqual(res['sources_saldo'].index.tolist(), ['Assets:Current Assets:Cash EUR', 'Assets:Current Assets:PostFinance:Private', 'Assets:Current Assets:PostFinance:Private-Bills', 'Liabilities:Credit Card CFF Visa'])
                                 
suite = TestLoader().loadTestsFromTestCase(IntegrTestBook)
TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 98.349s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [459]:
accs_cf = ['Clothes & Shoes']
res = book.reportCf(accs_cf, GncDate("2016-09-28"), GncDate("2017-04-27"))
print(res['gnc_period_begin'])
print(res['gnc_period_end'])
print(res['cf_total_inflow'])
print(res['cf_total_outflow'])
print(res['period_saldo'])
print(res['fx_rates'])
print(res['sinks_saldo'].value.tolist())
print(res['sinks_saldo'].quantity.tolist())
print(res['sinks_saldo'].index.tolist())
print(res['sources_saldo'].value.tolist())
print(res['sources_saldo'].quantity.tolist())
print(res['sources_saldo'].index.tolist())

20160927220000
20170426220000
1786.9379999999999
395.4600000000001
1391.4779999999998
{'base': 'CHF', 'EUR': (108660000, 100000000)}
[98.9, 9.9, 88.4, 39.8, 22.2, 2.7, 109.86000000000001, 23.7]
['Assets:Current Assets:PostFinance:Private-Bills', 'Expenses:Alcohol', 'Expenses:Baby:Clothes', 'Expenses:Baby:Toys', 'Expenses:Cosmetics', 'Expenses:Equipment', 'Expenses:Groceries', 'Expenses:Hobbies, Sport, Tourism']
[32.598, 368.2, 301.95, 1084.19]
['Assets:Current Assets:Cash EUR', 'Assets:Current Assets:PostFinance:Private', 'Assets:Current Assets:PostFinance:Private-Bills', 'Liabilities:Credit Card CFF Visa']


In [ ]:
class Currency(object):
    def __init__(self, val):
        self.value_f = 

In [ ]:
acc= [ ']